In [8]:
import ollama
import time

def test_mistral_connection():
    try:
        models = ollama.list()
        if not any(m.model == 'mistral:7b-instruct-v0.2-q4_0' for m in models.models):
            print("Модель не найдена. Выполните: ollama pull mistral:7b-instruct-v0.2-q4_0")
            return False
        print("Модель mistral:7b-instruct-v0.2-q4_0 готова к работе")
        return True
    except Exception as e:
        print(f"Ошибка подключения: {str(e)}")
        print("Убедитесь, что Ollama запущен (ollama serve)")
        return False

def query_llm(question: str, context: str = "") -> str:
    system_prompt = (
        "Ты — юридический ассистент Конституции Российской Федерации. "
        "Отвечай ИСКЛЮЧИТЕЛЬНО по предоставленному контексту. "
        "Если информация отсутствует в контексте — ответь точной фразой: "
        "'В Конституции Российской Федерации не указано'. "
        "НЕ придумывай данные. Цитируй конкретные статьи и пункты."
    )
    
    try:
        start = time.time()
        response = ollama.chat(
            model='mistral:7b-instruct-v0.2-q4_0',
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"КОНТЕКСТ:\n{context}\n\nВОПРОС:\n{question}"}
            ],
            options={
                'temperature': 0.1,  # Минимизация случайности генерации для юридической точности
                'num_ctx': 2048,     # Размер контекстного окна в токенах (оптимизация под 6GB VRAM)
                'top_p': 0.5         # Порог nucleus sampling для снижения риска галлюцинаций
            },
            keep_alive="5m"          # Сохранение модели в памяти GPU на 5 минут после последнего запроса
        )
        print(f"Время генерации: {time.time()-start:.1f} сек")
        return response['message']['content']
    
    except ollama.ResponseError as e:
        if "CUDA out of memory" in str(e):
            print("Ошибка VRAM: уменьшите num_ctx до 1024 или закройте приложения, использующие GPU")
        else:
            print(f"Ошибка Ollama: {str(e)}")
        raise SystemExit(1)
    
    except Exception as e:
        print(f"Непредвиденная ошибка: {str(e)}")
        raise

if __name__ == "__main__":
    if not test_mistral_connection():
        exit(1)
    
    print("\n" + "="*50)
    print("ТЕСТИРОВАНИЕ MISTRAL 7B (Q4_0)")
    print("="*50)
    
    print("\nТЕСТ 1: Вопрос без контекста")
    print("Вопрос: 'Какие обязанности у граждан РФ?'")
    answer = query_llm("Какие обязанности у граждан РФ?")
    print(f"Ответ:\n{answer}")
    
    print("\nТЕСТ 2: Вопрос со статьей 62")
    context = """
    СТАТЬЯ 62. ГРАЖДАНСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ
    1. Гражданин Российской Федерации не может быть лишен своего гражданства или права изменить его.
    2. Гражданин Российской Федерации не может быть выслан за пределы Российской Федерации или выдан другому государству.
    3. Российская Федерация гарантирует своим гражданам защиту и покровительство за ее пределами.
    """
    print("Вопрос: 'Может ли гражданин РФ быть выдан другому государству?'")
    answer = query_llm("Может ли гражданин РФ быть выдан другому государству?", context)
    print(f"Ответ:\n{answer}")
    
    print("\nТЕСТ 3: Проверка на галлюцинации")
    print("Вопрос: 'В какой статье Конституции прописаны штрафы за нарушение ПДД?'")
    answer = query_llm("В какой статье Конституции прописаны штрафы за нарушение ПДД?", context)
    print(f"Ответ:\n{answer}")
    
    print("\nВсе тесты пройдены успешно")

Модель mistral:7b-instruct-v0.2-q4_0 готова к работе

ТЕСТИРОВАНИЕ MISTRAL 7B (Q4_0)

ТЕСТ 1: Вопрос без контекста
Вопрос: 'Какие обязанности у граждан РФ?'
Время генерации: 22.0 сек
Ответ:
 В соответствии с ч. 1 ст. 65 Конституции Российской Федерации, граждане Российской Федерации имеют следующие обязанности:

а) соблюдать Конституцию и законы Российской Федерации;
б) охранять государственные символы – герб, флаг и Гимн Российской Федерации;
в) уважать права и свободы человека и гражданина;
г) содействовать развитию культуры, образованию, здравию, охране природы и историческому наследию;
д) участвовать в общественной жизни по способам, установленным законом;
е) выполнять военную повинность в соответствии с федеральными законами.

В Конституции Российской Федерации не указано обязанности граждан РФ платить налоги или поддерживать экономику страны (хотя статья 81 гласит: "Граждане Российской Федерации имеют право на социальное обеспечение в порядке, установленном федеральными законами"